In [19]:
library(tidyverse)
library(survey)

In [3]:
setwd("C:\\Users\\Elena.Mariani\\Documents\\Data\\National Diet Nutrition Survey\\UKDA-6533-tab\\tab")

*The aim of this activity is to ensure that we quality assure our understanding of the use of NDNS data but ensuring we can replicate published statistics. We will replicate parts of Tables 2.1, 3.2, 6.1 and 6.2. The tables are available to download on the [gov.uk website](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/943623/NDNS_from_years_1_to_9_data_tables__1_.zip)*

# Table 2.1: Total energy intake (kcal/day) and food energy intake (kcal/day) by  age and sex for waves 9-11

Person level diary data

In [11]:
pdd = read.table("ndns_rp_yr9-11a_personleveldietarydata_uk_20210831.tab", sep = "\t", header = T)
head(pdd)

,seriali,SurveyYear,NDays,AgeR,Sex,Country,TotalEMJ,FoodEMJ,EnergykJ,FoodEkJ,...,FOLICACID_CAPI,IRONONLYORWITHVITAMINC_CAPI,VITC_CAPI,OTHERNUTRIENTSUPPLEMENTS_CAPI,VITAMINSTWOORMOREINCLMULTIVITSNOMINERALS_CAPI,VITAMINSANDMINERALSINCLMULTIVITSMINERALS_CAPI,NONNUTRIENTSUPPLEMENTSINCLHERBAL_CAPI,SINGLEVITAMINSMINERALS_CAPI,MULTIVITAMINANDORMINERALSWITHOMEGA3_CAPI,SuppTaker_CAPI
,<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,100211281,10,4,37,2,England,4.486222,4.486222,4486.222,4486.222,...,0,0,0,0,0,0,0,0,0,0
2,100211282,10,4,14,1,England,3.345153,3.345153,3345.153,3345.153,...,0,0,0,0,0,0,0,0,0,0
3,100211131,10,4,45,2,England,6.092681,6.092681,6092.681,6092.681,...,0,0,0,0,0,0,0,0,0,0
4,100211133,10,4,11,1,England,7.794247,7.794247,7794.247,7794.247,...,0,0,0,0,0,0,0,0,0,0
5,100211021,10,4,50,2,England,6.290486,5.899001,6290.486,5899.001,...,0,0,0,0,0,0,0,0,0,0
6,100211024,10,4,10,2,England,5.383485,5.383485,5383.485,5383.485,...,0,0,0,0,0,0,0,0,0,0


Individual data

In [10]:
ind = read.table("ndns_rp_yr9-11a_indiv_20211020.tab", sep = "\t", header = T)
head(ind)

,seriali,serialh,Area,region,GOR,Addnum,surveyyr,AdChild,Quarter,month,...,wti_Y911,wtb_Y911,wtn_Y911,wtr_Y911,wtsu_Y911,wti_Y9,wtb_Y9,wtn_Y9,wtr_Y9,wtsu_Y9
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100101011,10010101,100101,4,11,1,10,1,4,3,...,1.8380267,1.5521456,1.7786253,1.343137,1.7773639,NA,NA,NA,NA,NA
2,100101022,10010102,100101,4,11,2,10,1,4,3,...,3.6760534,2.5689339,3.5373293,NA,3.5547278,NA,NA,NA,NA,NA
3,100101091,10010109,100101,4,11,9,10,1,4,3,...,3.3434320,NA,3.0121382,2.406434,3.1164039,NA,NA,NA,NA,NA
4,100101102,10010110,100101,4,11,10,10,1,4,3,...,2.9440746,2.1377580,2.8524036,2.043536,2.7950861,NA,NA,NA,NA,NA
5,100101154,10010115,100101,4,11,15,10,2,4,3,...,1.5349801,1.1662490,1.3174629,NA,1.6418613,NA,NA,NA,NA,NA
6,100101195,10010119,100101,4,11,19,10,2,4,3,...,0.7127176,0.7196654,0.6350292,NA,0.6612657,NA,NA,NA,NA,NA


Select variables of interest and merge with individual file to obtain weights.

In [20]:
pdd_ind <- merge(pdd[, c("seriali", "SurveyYear", "AgeR", "Sex", "Energykcal")],
                ind[, c("seriali", "surveyyr", "wti_Y911", "astrata1", "astrata2", "astrata3", "astrata4", "astrata5")],
                by.x = c("seriali", "SurveyYear"),
                by.y = c("seriali", "surveyyr"))

In [21]:
dim(pdd_ind)

[1] 3558   11

Specify the survey design

In [23]:
ndnsDesign <- svydesign(id = ~0,
                        strata  = ~astrata1,
                        weights = ~wti_Y911,
                        data    = pdd_ind)

## Children 1.5 - 3 years

In [31]:
ageChildren <- subset(ndnsDesign, AgeR > 0 & AgeR < 4)

In [32]:
svymean(~Energykcal, ageChildren, na.rm = TRUE)

           mean     SE
Energykcal 1056 16.595

In [35]:
svyquantile(~Energykcal, ageChildren, c(.025,.5,.975), ci=TRUE)

$Energykcal
       quantile   ci.2.5   ci.97.5       se
0.025  621.3323      NaN  683.7435      NaN
0.5   1040.9176 1015.905 1093.9567 19.74430
0.975 1537.5780 1519.549 1671.2395 38.37208

attr(,"hasci")
[1] TRUE
attr(,"class")
[1] "newsvyquantile"

## Adults 19-64 Years

In [36]:
ageAdult <- subset(ndnsDesign, AgeR > 18 & AgeR < 65)

In [37]:
svymean(~Energykcal, ageAdult, na.rm = TRUE)

             mean     SE
Energykcal 1828.2 19.207

In [38]:
svyquantile(~Energykcal, ageAdult, c(.025,.5,.975), ci=TRUE)

$Energykcal
       quantile    ci.2.5   ci.97.5       se
0.025  876.6278  800.5475  907.9056 27.35938
0.5   1764.0605 1732.6416 1803.9366 18.16897
0.975 3061.4611 2990.0078 3303.5826 79.91209

attr(,"hasci")
[1] TRUE
attr(,"class")
[1] "newsvyquantile"